#Introduction

In this notebook, we build a chatbot that lets users enter city names to look up weather forecast for the next 5 days. The weather forecast data is retrieved from the API at [www.visualcrossing.com/weather/weather-data-services](https://www.visualcrossing.com/weather/weather-data-services). An LLM model is also used to check if the input is valid, i.e. whether the user entered a city name.

#Weather API Retrieval

First, we define the function that retrieves weather information from the [API](https://www.visualcrossing.com/weather/weather-data-services) by taking a city name as the input.

In [1]:
#Define the function to retrieve weather from API
import pandas as pd
def API_weather(city):
  city = city.replace(' ', '%20').replace(',', '%2C')
  url = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}?unitGroup=metric&include=days&key=NRB7SJYH484EEMHW2W6KLT8YR&contentType=csv'
  weather_df = pd.read_csv(url).iloc[0:5][['name', 'datetime', 'tempmax', 'tempmin', 'temp', 'humidity', 'precipprob', 'conditions']]
  return weather_df

Below are some examples of what kind of information is retrieved.

In [2]:
API_weather('Ho Chi Minh City, Vietnam')

,name,datetime,tempmax,tempmin,temp,humidity,precipprob,conditions
0,"Hồ Chí Minh, Việt Nam",2024-09-02,29.4,24.0,25.7,88.3,100.0,"Rain, Partially cloudy"
1,"Hồ Chí Minh, Việt Nam",2024-09-03,29.2,24.0,25.2,90.1,100.0,"Rain, Overcast"
2,"Hồ Chí Minh, Việt Nam",2024-09-04,30.1,24.2,26.2,86.9,96.8,"Rain, Overcast"
3,"Hồ Chí Minh, Việt Nam",2024-09-05,29.5,24.9,26.2,86.4,93.5,"Rain, Overcast"
4,"Hồ Chí Minh, Việt Nam",2024-09-06,28.8,24.6,26.1,86.2,93.5,"Rain, Overcast"


In [3]:
API_weather('Tokyo, Japan')

,name,datetime,tempmax,tempmin,temp,humidity,precipprob,conditions
0,"Tokyo, Japan",2024-09-02,32.9,27.4,29.8,68.9,41.9,Partially cloudy
1,"Tokyo, Japan",2024-09-03,27.7,23.3,24.6,84.1,96.8,"Rain, Partially cloudy"
2,"Tokyo, Japan",2024-09-04,28.4,23.2,25.2,75.0,64.5,"Rain, Partially cloudy"
3,"Tokyo, Japan",2024-09-05,30.4,24.2,26.9,70.7,19.4,Partially cloudy
4,"Tokyo, Japan",2024-09-06,33.7,26.4,29.2,63.4,12.9,Partially cloudy


In [4]:
API_weather('Lausanne, Switzerland')

,name,datetime,tempmax,tempmin,temp,humidity,precipprob,conditions
0,"Lausanne, VD, Suisse",2024-09-01,30.3,17.4,22.8,72.2,100.0,"Rain, Partially cloudy"
1,"Lausanne, VD, Suisse",2024-09-02,25.0,20.5,22.2,82.8,100.0,"Rain, Overcast"
2,"Lausanne, VD, Suisse",2024-09-03,24.7,20.7,22.2,82.3,100.0,"Rain, Overcast"
3,"Lausanne, VD, Suisse",2024-09-04,22.4,20.0,20.9,85.2,100.0,"Rain, Overcast"
4,"Lausanne, VD, Suisse",2024-09-05,20.6,17.1,19.1,84.7,83.9,"Rain, Overcast"


In [5]:
API_weather('London, UK')

,name,datetime,tempmax,tempmin,temp,humidity,precipprob,conditions
0,"London, England, United Kingdom",2024-09-01,26.9,16.6,21.1,77.0,100.0,"Rain, Partially cloudy"
1,"London, England, United Kingdom",2024-09-02,23.3,18.4,20.4,81.5,32.3,Partially cloudy
2,"London, England, United Kingdom",2024-09-03,21.0,16.5,18.3,72.6,22.6,Partially cloudy
3,"London, England, United Kingdom",2024-09-04,19.9,12.9,16.1,67.6,45.2,"Rain, Partially cloudy"
4,"London, England, United Kingdom",2024-09-05,17.0,12.6,14.9,78.7,45.2,"Rain, Overcast"


In [6]:
API_weather('Austin, Texas')

,name,datetime,tempmax,tempmin,temp,humidity,precipprob,conditions
0,"Austin, TX, United States",2024-09-01,32.8,24.2,28.3,72.0,34.0,Partially cloudy
1,"Austin, TX, United States",2024-09-02,30.6,23.4,26.5,85.5,65.0,"Rain, Partially cloudy"
2,"Austin, TX, United States",2024-09-03,29.5,22.8,25.6,88.3,56.0,"Rain, Partially cloudy"
3,"Austin, TX, United States",2024-09-04,29.4,22.8,25.7,83.9,55.0,"Rain, Partially cloudy"
4,"Austin, TX, United States",2024-09-05,30.2,23.2,26.3,83.4,25.0,Partially cloudy


## LLM for Checking Valid Input

We will use an LLM model to check whether a valid city name is given as the input. First, we install the necessary dependencies.

In [7]:
!pip install --quiet --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 117.8 MB/s eta 0:00:00


For the next step, you will need to enter your Hugging Face token to log into Hugging Face. Please log into your Hugging Face account or sign up for one and then go to [huggingface.co/settings/tokens](https://huggingface.co/settings/tokens) to create an access token with write permission. When running the code cell below, please enter "n" when prompted "Add token as git credential?".

In [8]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In this notebook, we use the LLM model "meta-llama/Meta-Llama-3.1-8B-Instruct," which is a gated model. Please request access to the model at [huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct) before running the code cell below.

In [9]:
#Download the LLM model
import transformers
import torch

model_id = 'meta-llama/Meta-Llama-3.1-8B-Instruct'

pipeline = transformers.pipeline(
    'text-generation',
    model=model_id,
    model_kwargs={'torch_dtype': torch.bfloat16},
    device_map='auto',
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [10]:
#Initialize the LLM model as a valid input checker through prompting
messages = [
    {'role': 'system', 'content':
     '''Act as a valid input checker. Check whether the prompt contains a valid city name.
     If yes, return the city name in its original format including the country name if provided.
     If no, return "None". Do not return anything else. Note that a country name does not count
     as a valid city name.'''},
    {'role': 'user', 'content': 'prompt'},
]

The LLM model will return the city name if one is provided and "None" if no valid city name is provided.

In [11]:
#An example with a valid input
input_1 = 'Ho Chi Minh City, Vietnam'
messages[1]['content'] = input_1
check_1 = pipeline(
    messages,
    max_new_tokens=256,
)[0]['generated_text'][-1]['content']
print(check_1)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Ho Chi Minh City, Vietnam


In [12]:
#An example with an invalid input
input_2 = 'France'
messages[1]['content'] = input_2
check_2 = pipeline(
    messages,
    max_new_tokens=256,
)[0]['generated_text'][-1]['content']
print(check_2)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


None


In [13]:
#Another example with an invalid input
input_3 = 'What time is it?'
messages[1]['content'] = input_3
check_3 = pipeline(
    messages,
    max_new_tokens=256,
)[0]['generated_text'][-1]['content']
print(check_3)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


None


Following the process of the examples above, we define the function that uses the LLM model to check for valid inputs.

In [14]:
#Define the function to check whether the input is a valid city name
def valid_city_check(input):
  messages[1]['content'] = input
  check = pipeline(
      messages,
      max_new_tokens=256,
  )[0]['generated_text'][-1]['content']
  return check

#Gradio Interface Deployment

Finally, we use Gradio to deploy a chat interface that lets users enter a city name to get a five-day weather forecast. The chat interface also contains a button that displays some information about the chatbot when clicked.

In [15]:
#Install and import Gradio
!pip install --upgrade --quiet gradio
import gradio as gr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 116.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.1 MB/s eta 0:00:00


In [16]:
#Define the function that processes the messages/inputs on Gradio interface
def weather_app(message, history):
  #First, check for valid input
  check = valid_city_check(message)
  if check == 'None':
    return 'Please enter a valid city name.'
  else:
    #Retrieve weather forecast and return the results
    weather_df = API_weather(check)
    weather_for_each_day = []
    for index, row in weather_df.iterrows():
      weather_for_each_day.append(
          f'''<b>{row['datetime']}</b>\n
          {row['conditions']}\n
          Mean Temperature: {row['temp']}°C\n
          Min Temperature: {row['tempmin']}°C\n
          Max Temperature: {row['tempmax']}°C\n
          Humidity: {row["humidity"]}%\n
          Precipitation: {row['precipprob']}%'''
          )
    return f'Here is the weather forecast for {check} for the next five days:\n'+'\n'.join(weather_for_each_day)

In [17]:
#Define the function that loads when the info button is clicked
def load():
  return [('Some info about this chatbot',
          '''Author of chatbot: Duc Tran\n
          This weather chatbot was made as part of the technical assessment for an internship role at PM Accelerator.\n
          The PM Accelerator Program is designed to support PM professionals through every stage of their
          career. From students looking for entry-level jobs to Directors looking to take on a leadership role, our
          program has helped over hundreds of students fulfill their career aspirations.\n
          Our PM Accelerator community are ambitious and committed. Through our program they have learnt,
          honed and developed new PM and leadership skills, giving them a strong foundation for their future endeavours.''')]

In [18]:
with gr.Blocks() as demo:
  #Home screen of the chatbot
  chatbot = gr.Chatbot(placeholder='<strong><big>Please enter a city name to look up the weather</big></strong>')
  #Chat interface
  gr.ChatInterface(fn=weather_app,
                   chatbot=chatbot,
                   examples=['Ho Chi Minh City, Vietnam', 'Tokyo, Japan', 'Lausanne, Switzerland', 'London, UK', 'Austin, Texas'],
                   title='Weather Chatbot')
  #Info button
  gr.Button('Info about this chatbot').click(fn=load, inputs=None, outputs=chatbot)

#Launch the Gradio interface
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://361a7c98cb7b6d543a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
